Marcin Wardyński  
czwartek, 8:00

### Przygotowanie danych oraz funkcji pomocniczych

Zaczynamy od załadowania tysiąca losowych pasaży ze zbioru FIQA-PL:

In [1]:
from datasets import load_dataset
seed = 7
passage_number = 1000

corpus = load_dataset("clarin-knext/fiqa-pl", name="corpus")
passages = corpus['corpus'].shuffle(seed=seed).select(range(passage_number))['text']

/Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Poniższy kod zawiera zapytania do LLM `Mistral` o siedmiu miliardach parametrów, który został uruchomiony w ramach środowiska `Ollama`. Ponieważ zapytania do LLM trwają kilka sekund, odpowiedzi na prompty zostały dodatkowo zachowane w cache-u na dysku, co w znacznym stopniu przyśpiesza późniejsze polecenia.

(Utworzyłem też funkcję korzystającą z cache w pamięci, lecz z niej nie korzystam, gdyż cache na dysku w zupełności wystarcza).

In [160]:
import requests
import hashlib
from diskcache import Cache
from functools import lru_cache

OLLAMA_SERVER_URL = "http://localhost:11434"
cache = Cache("ollama_cache")
model_mistral = "mistral"

@lru_cache(maxsize=1100)
def ask_ollama_with_memory_cache(model, prompt):
    return ask_ollama_with_disk_cache(model, prompt)

def ask_ollama_with_disk_cache(model, prompt):
    
    cache_key = hashlib.sha256((model+prompt).encode()).hexdigest()
    
    if cache_key in cache:
        answer = cache[cache_key]
        return (1, answer)
    else:
        answer = ask_ollama_server(model, prompt)
        cache[cache_key] = answer
        return (0, answer)


def ask_ollama_server(model, prompt):
    url = f"{OLLAMA_SERVER_URL}/api/generate"
    
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "options": {"num_ctx": 8192}
    }

    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        data = response.json()
        return data["response"]

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None


In [46]:
!python -m spacy download pl_core_news_sm

Python(73480) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 20.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')


Funkcja `find_named_entities_spacy()` przeprowadza operację NER używając pakietu `SpaCy`:

In [100]:
import pl_core_news_sm

nlp = pl_core_news_sm.load()

def find_named_entities_spacy(passage):
    results = {"persName": [],
                "orgName": [],
                "placeName": [],
                "geogName": [],
                "date": [],
                "time": []}

    doc = nlp(passage)
    for ent in doc.ents:
        results[ent.label_].append(str(ent))
    
    
    return results

### Prompt Engineering

Poniższe elememty składowe prompta są wynikiem kilku iteracji dopracowywania jego treści. Jego ważnymi elementami w wydaniu dla zapytani `zero-shot` są:
- jasne polecenie zadania podstawowego
- wykaz klas do uwzględnienia przy klasyfikacji
- nazwanie formatu wyjściowego, JSON
- sposób potraktowania przypadków szczególnych, np: pusta lista dla klasy bez elemntów
- powtórzenie ogólnych wymagań dotyczących formatu wyjściowego
- dokładna specyfikacja formatu wyjściowego
- jasna specyfikacja sekcji polecenia, formatu wyjściowego, paragrafu do przetworzenia oraz oczekiwanego wyniku

In [254]:
prompt_intro = "Przeanalizuj podany tekst i wypisz nazwy własne, w odmianie w jakiej występują w tekście, podzielone na kategorie: osoby, organizacje, nazwy administracyjne miejsc, nazwy geograficzne, daty, czas. Twoja odpowiedź powinna składać się wyłącznie z obiektu JSON o formacie zdefiniowanym poniżej. Jeśli pewna kategoria nie ma odnalezionych reprezentantów, zwróć dla niej pustą listę. Upewnij się, że wynik jest czysty i zawiera wyłącznie obiekt JSON, bez żadnych komentarzy!\n"
prompt_json_format = "Format JSON:\n{\"persName\": [\"lista osób\"],  \"orgName\": [\"lista organizacji\"], \"placeName\": [\"lista nazw administracyjnych miejsc\"], \"geogName\": [\"lista nazw geograficznych\"], \"date\": [\"lista dat\"], \"time\": [\"lista określeń czasu\"]}\n"
prompt_output = "Wynik:"

In [282]:
import json
import time

def perform_zero_shot_prompt(passage):
    prompt_input = f"Tekst wejściowy:\n\"{passage}\"\n"
    prompt = prompt_intro + prompt_json_format + prompt_input + prompt_output
        
    result = ask_ollama_with_disk_cache(model_mistral, prompt)[1].replace('\n', '')
    try:
        return json.loads(result)
    except json.JSONDecodeError as e:
        pass
    
    
start_time = time.time()

for i, passage in enumerate(passages):
    print(i, end="\n" if (i+1) % 25 == 0 else " ")
    perform_zero_shot_prompt(passage)

elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time}s")


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74
75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99
100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124
125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149
150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174
175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199
200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249
250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274
275 276 27

Dodatkowo z wyniku LLM wycinane są znaki nowej linii `\n`, które przeszkadzają w parsowaniu formatu JSON.

Pomimo jasnej specyfikacji oczekiwanego formatu, dla pasażów 280 oraz 753 LLM zwraca wynik w złym formacie:
- `' {"persName": ["Client"], "orgName": ["facet", "lowball", "luxury price company"]}   {"placeName": [], "geogName": []}   {"date": [], "time": []}`
- `' {      "persName": ["sam"],      "orgName": [],      "placeName": [],      "geogName": [],      "date": [],      "time": ["kilka godzin", "rok", "oteł"] // OTEL jest czasem polskim oznaczającym okres (w tym przypadku rok)   }'`

W pierwszym przypadku obiekt JSON został zamknięty już za nazwami organizacji, po czym ponownie otwarty. Natomiast dla drugiego pasażu LLM pomimo jasnych wskazówek dodał komentarz, który uniemożliwił parsing.

Obydwa przypadki zostały odrzucone i biorą udziału w dalszym teście.

Przy zapytaniu `few-shot` zapytanie `zero-shot` jest uzupełniane o kolejną jasno zdefiniowaną sekcję: sekcję przykładów. Poza nią została dodana kolejna wskazówka dla modelu, który zaczął w kilku odpowiedziach pomijać `:` dla klasy `time`.

In [284]:
example1_text = "Czy możesz mi pokazać gdzie? Ponieważ z pewnością, kiedy Obama nałożył podatek na finansowanie ACA, gospodarka rozkwitła, zatrudniono milion lekarzy, setki szpitali i tysiące spacerów w klinikach, które zbudowaliśmy. A giełda rozwijała się bardziej."
example1_answer = "{\"persName\": [\"Obama\"],  \"orgName\": [\"ACA\"], \"placeName\": [], \"geogName\": [], \"date\": [], \"time\": []}"

example2_text = "„Postęp komputerowy w coraz większym stopniu pozwala producentom dostosowywać zamówienia i szybciej wysyłać towary. W nowym świecie wytwarzanie produktów w odległych krajach o niskich płacach, takich jak Chiny, może być wadą: wysyłanie gotowych produktów do Stanów Zjednoczonych może trwać zbyt długo – tygodnie, miesiące. „Chodzi o bliskość z klientem” – powiedział Michael Mandel, główny strateg ekonomiczny w Progressive Policy Institute. „„Zdobędziesz trwałą i trwałą przewagę nad konkurencją zagraniczną”. który okupował Biały Dom. Mimo to prezydent Donald Trump skorzystał z okazji, by w środę wziąć udział w ogłoszeniu przez Foxconna, mówiąc, że „zdecydowanie” wydarzyło się z powodu jego wyboru i dążenia do cięć podatkowych i regulacyjnych. [źródło] ://hosted.ap.org/dynamic/stories/U/US_BC_US_AMAZON_AND_FOXCONN_SPEED_TO_CUSTOMERS?SITE=AP&SECTION=HOME&TEMPLATE=DEFAULT&CTIME=2017-07-27-03-16-36)\""
example2_answer = "{\"persName\": [\"Michael Mandel\", \"Donald Trump\"],  \"orgName\": [\"Progressive Policy Institute\", \"Biały Dom\", \"Foxconna\"], \"placeName\": [\"Chiny\", \"Stanów Zjednoczonych\"], \"geogName\": [], \"date\": [], \"time\": []}"

example3_text = "„Płać za grę? Masz na myśli, że płacą za mocniejsze źródła danych, których inne firmy nie potrzebują, na pewno tak. Ale każdy może teraz handlować na giełdach amerykańskich (NYSE, NASDAQ, BATS itp.), co nie było prawdą 20 lat temu kiedy NYSE miała specjalistyczny monopol, więc tak, rynki są bardziej demokratyczne niż kiedykolwiek. Uwaga na marginesie, giełdy czerpią bezpośrednie zyski ze zwiększonego wolumenu obrotu, ponieważ przyjmują niewielki procent każdej transakcji, więc nie jestem pewien, co to znaczy, że nie czerpią z tego bezpośrednich korzyści. Mam też wrażenie, że nie rozumiesz zakresu działalności HFT, szczytowe zyski HFT były rzędu 7 miliardów w czasie największego wolumenu w ostatniej dekadzie (2005-2010 ). Są teraz około 1B. W porównaniu z bilionami dolarów, które zmieniają właściciela w ciągu roku, jest to kiepska wymiana, trudno nazwać „kran z darmowymi pieniędzmi”. Również Katsayuma otworzyła kolejną ciemną pulę, która obsługuje duże wolumeny klientów (twoich goldmans i merrils) Jedyną różnicą w IEX jest to, że ma darmową kampanię marketingową ign, aby przyciągnąć klientów. Poważnie, IEX jest niczym innym jak istniejącymi stałymi, krzyżowymi ciemnymi basenami, które przy okazji wkurzają inwestorów detalicznych bardziej niż oświetlone giełdy, takie jak NASDAQ. Katsayuma został zmiażdżony w swoich egzekucjach, ponieważ nie mógł nadążyć z czasem, a potem trafił w dziesiątkę, namawiając Michaela Lewisa, by namalował go jako „bohatera”, szczerze mówiąc, jestem zdziwiony, jakie szczęście miał ten koleś. Dealerzy brokerów BTW otrzymują preferencyjne traktowanie w IEX, co oznacza, że ​​mogą ciąć przed inwestorami detalicznymi. Dlaczego tak się na to zastanawiasz, czy wykonałeś kilka złych transakcji na eTrade i potrzebujesz kozła ofiarnego?”"
example3_answer = "{\"persName\": [\"Katsayuma\", \"Katsayuma\", \"Michaela Lewisa\"],  \"orgName\": [\"NYSE\", \"NASDAQ\", \"BATS\", \"NYSE\", \"HFT\", \"goldmans\", \"merrils\", \"IEX\", \"ign\", \"IEX\", \"NASDAQ\", \"IEX\", \"eTrade\"], \"placeName\": [], \"geogName\": [], \"date\": [\"2005\", \"2010\"], \"time\": []}"
example2_full = f"Przykład 3:\n\"{example2_text}\"\nOdpowiedź do przykładu 1:\n\"{example2_answer}\"\n"

example4_text = "Nie zaczynaj od inwestowania w kilka pojedynczych firm. To ryzykowne. Chcesz przykładu? Myślę o dużej firmie, powiedzmy około 120 miliardów dolarów, znanej firmie i dobrych stałych dywidendach. Radzili sobie całkiem nieźle i generalnie byli zajęci przekonywaniem ludzi, że patrzą w przyszłość z nowymi, przyjaznymi dla środowiska technologiami. Potem... poszli i wylali garść ropy do Zatoki Meksykańskiej. Tak, to nie był ładny obraz, gdyby BP było tego dnia jedną z pięciu spółek w twoim portfelu. Sprawy wyglądałyby jednak znacznie lepiej, gdyby byli jedną z 500 lub 5000 firm. Więc. Po pierwsze, dążyć do dywersyfikacji za pośrednictwem funduszy inwestycyjnych lub ETF. (Osobiście uważam, że prawdopodobnie powinieneś zacząć od funduszy wzajemnych: po pierwsze unikasz opłat transakcyjnych. Łatwiej jest również umieścić średnie kwoty w dolarach w funduszach niż w ETF-ach, nawet jeśli otrzymujesz wolny od opłat handel ETF-ami. może dać ci lepsze wskaźniki wydatków, ale im mniej pieniędzy zainwestowałeś, tym mniej ważne.) Gdy masz przyzwoity portfel – dziesiątki tysięcy dolarów lub więcej – możesz zacząć rozważać posiadanie akcji poszczególnych spółek. Zwróć uwagę na opłaty, w tym opłaty transakcyjne / prowizje. Jeśli kupisz akcje o wartości 2000 USD i zapłacisz 20 USD prowizji, stracisz już 1%. Jeśli trzymasz fundusz powierniczy lub ETF, spójrz na wskaźnik wydatków. Roczna realna stopa zwrotu na giełdzie wynosi około 4%. (Prawdziwy zwrot jest po uwzględnieniu inflacji.) Jeśli Twoja opłata wynosi 1%, to około jedna czwarta Twoich zarobków, co jest ogromne. I chociaż funduszowi powierniczemu łatwo jest od czasu do czasu przewyższyć rynek o 1%, to naprawdę ciężko jest robić to konsekwentnie. Kiedy już przyjrzysz się poszczególnym firmom, powinieneś przeprowadzić wiele nieznośnych, nudnych, głupich poszukiwań, a nie kupować tylko akcji na podstawie ich marki. Będziesz zainteresowany kilkoma danymi. Głównym z nich jest prawdopodobnie wskaźnik P/E (cena/zysk). Jeśli weźmiesz odwrotność tego, uzyskasz wskaźnik, w jakim Twoja inwestycja przyniesie Ci pieniądze (np. P/E 20 to 5%, P/E 10 to 10%). Jeśli wszystko inne jest równe, niższy wskaźnik P/E jest dobrą rzeczą: oznacza to, że kupujesz dochód firmy naprawdę tanio. Jednak wszystko inne rzadko jest równe: jeśli akcje są naprawdę tanie, zwykle dzieje się tak dlatego, że inwestorzy nie myślą, że mają przed sobą długą przyszłość. Zarobki nie zawsze są spójne. Istnieje wiele innych miar, takich jak beta (korelacja z ogólnym rynkiem: bardziej ryzykowne akcje niestabilne mają wyższe wartości), marże brutto, cena do nielewarowanych wolnych przepływów pieniężnych i tym podobne. Ponownie wykonaj nudne badania, w przeciwnym razie po prostu grasz w gry ze swoimi pieniędzmi."
example4_answer = "{\"persName\": [],  \"orgName\": [\"BP\"], \"placeName\": [], \"geogName\": [\"Zatoki Meksykańskiej\"], \"date\": [], \"time\": []}"

example5_text = "„To najlepszy tl;dr, jaki mogłem zrobić, [oryginał](https://www.bloomberg.com/news/articles/2017-08-22/hong-kong-braces-for-storm-hato-stock- handel może zostać zakłócony) zmniejszony o 73% (jestem botem) ***** > Hongkong podniósł poziom ostrzeżenia przed burzą do najwyższego poziomu po raz pierwszy od pięciu lat i odwołał poranną sesję handlową jako Poważna Tajfun Hato zbliżył się do centrum finansowego.> Jeśli sygnał będzie obowiązywał do południa, handel na czwartym co do wielkości rynku akcji na świecie zostanie dziś zlikwidowany, zgodnie z zasadami Hong Kong Exchanges & Clearing Ltd. > O godzinie 9 rano ciężki tajfun Hato znajdował się około 80 kilometrów na południe od Hongkongu, podało Obserwatorium.***** [**Rozszerzone podsumowanie**](http://np.reddit.com/r/autotldr/ komentarze/6vgq2t/hong_kong_delays_morning_trading_as_typhoon_hato/) | [FAQ](http://np.reddit.com/r/autotldr/comments/31b9fm/faq_autotldr_bot/ \"\"Wersja 1.65, ~196625 tl\");[do tej pory drs.\" Opinia](http://np.reddit.com/message/compose?to=%23autotldr \"\"PM i komentarze są monitorowane, mile widziane są konstruktywne opinie.\"\") | *Najlepsze* *Słowa kluczowe*: **Hong**^#1 **Kong**^#2 **Zamknij**^#3 **Tajfun**^#4 **Handel**^#5\""
example5_answer = "{\"persName\": [],  \"orgName\": [\"Hong Kong Exchanges & Clearing Ltd.\"], \"placeName\": [\"Hongkong\", \"Hongkongu\"], \"geogName\": [], \"date\": [], \"time\": [\"9 rano\"]}"

examples_text = [example1_text, example2_text, example3_text, example4_text, example5_text]
examples_answer = [example1_answer, example2_answer, example3_answer, example4_answer, example5_answer]

additional_hint = "Nie zapominaj o znaku \":\" pomiędzy \"time\" i \"[\"\n"

examples_snippet = []
for i in range(len(examples_text)):
    examples_snippet.append(f"Przykład {i+1}:\n{examples_text[i]}\nOdpowiedź do przykładu {i+1}:\n{examples_answer[i]}\n")

Ponieważ prompt z przykładami potrafi dojść do prawie 7 000 tokenów, limit `Ollama` musiał zostać podniesiony z 2048 tokenów do 8192 tokenów, co jest maksymalnym rozmiarem kontekstu dla użytego LLM: Mistral

In [292]:
def perform_few_shot_prompt(passage):
    prompt_input = f"Tekst wejściowy:\n\"{passage}\"\n"
    prompt = prompt_intro + prompt_json_format + "".join(examples_snippet) + additional_hint + prompt_input + prompt_output
        
    result = ask_ollama_with_disk_cache(model_mistral, prompt)[1].replace('\n', '').replace('"time[]"', '"time": []')
    try:
        return json.loads(result)
    except json.JSONDecodeError as e:
        pass
    
start_time = time.time()

for i, passage in enumerate(passages):
    print(i, end="\n" if (i+1) % 25 == 0 else " ")
    perform_few_shot_prompt(passage)

elapsed_time = time.time() - start_time
print(f"\nElapsed time: {elapsed_time}s")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74
75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99
100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124
125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149
150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174
175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199
200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249
250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274
275 276 27

Częstym błędem w formacie odpowiedzi na prompt `few-shot` był zły zapis klasy `time` z pominięciem dwukropku. Błąd nie został wyeliminowany przez LLM po dodaniu dodatkowego polecenia z tym związanego, więc usuwam ten błąd już po stronie mojej funkcji kodem `result.replace('"time[]"', '"time": []')`

### Obliczanie metryk

Poniższy zestaw funkcji oblicza metryki `precision`, `recall` oraz `f1-score` dla operacji NER wykonanej dwoma metodami, z tym że wynik pierwszej z metod traktowany jest jako baseline.

Metryki nie są uśrednianie dla wartości per pasaż, lecz zbierane kumulatywnie i obliczone dla całego zbioru wypróbowanych pasaży. Podjąłem decyzję o takim obliczaniu metryk, ponieważ nie interesują nas ich wartości dla poszczególnych dokumentów, lecz dla całego zbioru. Poza tym, rozkład nazw własnych wśród dokumentów nie jest zrównoważony, przez co uśrednianie wyników obliczonych per dokument tylko zaciemniłoby faktyczną jakość klasyfikacji nazw własnych.

Ponieważ występuje wiele nieścisłości w przygotowanych danych oraz sposobie wyznaczania nazw własnych, przygotowałem dwa mechanizmy porównujące obecność jednej nazwy własnej w drugim zbiorze. Jedna bazuje na dokładnym dopasowaniu, podczas gdy druga zwraca wyniki również dla operacji zawierania sprawdzonej w obydwu kierunkach.

Do porównywania wyników poszczególnych metod użyję tylko tej bardziej wymagającej metody, natomiast dodatkowe poluzowanie pokazuje nam ewentualne przyrost jakości wyników, gdyby dane były przygotowane w lepszy sposób, lub też model lepiej odnajdował nazwy własne. Oczywiście wyniki drugiej metody należy traktować jedynia jako pewne przybliżenie, a poniższe wyniki zawierają przykłady opisujące różne przypadki niedopasowania.

In [318]:
def get_corr_ent_strict(other_ent, base_ents):
    return other_ent if other_ent in base_ents else None

def get_corr_ent_not_strict(other_ent, base_ents):
    for base_ent in base_ents:
        if other_ent in base_ent or base_ent in other_ent:
            return base_ent
    return None

def calculate_conf_matrix(baseline_entities, other_entities, strict):
    base_ents_tmp = baseline_entities[:]
    other_ents_tmp = other_entities[:]

    tp_list = []

    for other_ent in other_entities:
        base_ent = get_corr_ent_strict(other_ent, base_ents_tmp) if strict else get_corr_ent_not_strict(other_ent, base_ents_tmp)
        if base_ent:
            base_ents_tmp.remove(base_ent)
            other_ents_tmp.remove(other_ent)
            tp_list.append(other_ent)

    tp = len(tp_list)
    fp = len(other_ents_tmp)
    fn = len(base_ents_tmp)

    return tp, fp, fn

def calculate_precision_recall_f1_score(conf_matrix_by_class):
    results = {}
    for entity_class, conf_matrix in conf_matrix_by_class.items():
        results[entity_class] = {}

        if conf_matrix["TP"]+conf_matrix["FP"] != 0:
            precision = conf_matrix["TP"]/(conf_matrix["TP"]+conf_matrix["FP"])

        else:
            precision = 0
        results[entity_class]["precision"] = precision


        if conf_matrix["TP"]+conf_matrix["FN"] != 0:
            recall = conf_matrix["TP"]/(conf_matrix["TP"]+conf_matrix["FN"])
        else:
            recall = 0
        results[entity_class]["recall"] = recall

        if precision + recall != 0:
            f1_score = 2 * (precision * recall) / (precision + recall)
        else:
            f1_score = 0
        results[entity_class]["f1_score"] = f1_score
    
    return results

def calculate_metrics(passages, baseline_fun, other_fun, strict=True):
    conf_matrix_by_class = {"persName": {"TP": 0, "FP": 0, "FN": 0},
                                "orgName": {"TP": 0, "FP": 0, "FN": 0},
                                "placeName": {"TP": 0, "FP": 0, "FN": 0},
                                "geogName": {"TP": 0, "FP": 0, "FN": 0},
                                "date": {"TP": 0, "FP": 0, "FN": 0},
                                "time": {"TP": 0, "FP": 0, "FN": 0},
                                "total": {"TP": 0, "FP": 0, "FN": 0}}
    
    for passage in passages:
        results_baseline = baseline_fun(passage)
        results_other = other_fun(passage)

        if results_other is None:
            continue

        for entity_class in results_baseline.keys():
            if entity_class not in results_other.keys():
                continue 

            baseline_entities = results_baseline[entity_class]            
            other_entities = results_other[entity_class]

            tp, fp, fn = calculate_conf_matrix(baseline_entities, other_entities, strict)

            conf_matrix_by_class[entity_class]["TP"] += tp
            conf_matrix_by_class["total"]["TP"] += tp
            conf_matrix_by_class[entity_class]["FP"] += fp
            conf_matrix_by_class["total"]["FP"] += fp
            conf_matrix_by_class[entity_class]["FN"] += fn
            conf_matrix_by_class["total"]["FN"] += fn

    return calculate_precision_recall_f1_score(conf_matrix_by_class)

Poniżej rozpoczynam badanie jakości wyników dla wybranych pasaży z korpusu FIQA-PL.

W poprzednim laboratorium mogliśmy zaobserwować, że klasyfikacja nazw własnych dostarczona przez NER od SpaCy nie jest idealna, a wręcz jest ona od ideału daleka, gdyż jest to mały model stosujący podejście naiwne. Stąd też nie należy od poniższych wynikach sporo oczekiwać.

Może dziwić decyzja użycia wyniów ze SpaCy jako baseline-u do oceny LLM, więc poniższe wyniki traktuję tylko jako ciekawostkę, jak może wyglądać w pełni zautomatyzowany pomiar jakości operacji NER, bez udziału człowieka. Przy odpowiedzi na pytania do laboratorium skupię się głównie na dalszej sekcji, gdzie to człowiek ingeruje w ocenę jakości operacji NER.

In [319]:
calculate_metrics(passages, find_named_entities_spacy, perform_zero_shot_prompt, True)

{'persName': {'precision': 0.15578947368421053,
  'recall': 0.09585492227979274,
  'f1_score': 0.11868484362469928},
 'orgName': {'precision': 0.18989071038251365,
  'recall': 0.15010799136069114,
  'f1_score': 0.16767189384800965},
 'placeName': {'precision': 0.22422680412371135,
  'recall': 0.12305516265912306,
  'f1_score': 0.1589041095890411},
 'geogName': {'precision': 0.014925373134328358,
  'recall': 0.016,
  'f1_score': 0.015444015444015444},
 'date': {'precision': 0.20754716981132076,
  'recall': 0.1896551724137931,
  'f1_score': 0.1981981981981982},
 'time': {'precision': 0.0043859649122807015,
  'recall': 0.3333333333333333,
  'f1_score': 0.008658008658008658},
 'total': {'precision': 0.15998155832180727,
  'recall': 0.12549728752260397,
  'f1_score': 0.1406566680178354}}

|           | persName | orgName | placeName | geogName | date   | time   | **total**  |
|-----------|----------|---------|-----------|----------|--------|--------|------------|
| precision | 0.1557   | 0.1898  | 0.2242    | 0.0149   | 0.2075 | 0.0043 | **0.1599** |
| recall    | 0.0958   | 0.1501  | 0.1230    | 0.0160   | 0.1896 | 0.3333 | **0.1254** |
| f1_score  | 0.1186   | 0.1676  | 0.1589    | 0.0154   | 0.1981 | 0.0086 | **0.1406** |

Powyższa tabela przedsawia wyniki metryk `precision`, `recall` oraz `f1-score` w ujęciu ogólnym oraz dla każdej kategorii z osobna dla wywołania `zero-shot`, używając SpaCy jako wyników referencyjnych.

Wyniki prezentują się dość słabo, `precision` dla wszystkich wyników wynosi zaledwie 0.1599, natomiast `recall` jeszcze mniej: 0.1254, a więc i `f1-score` nie jest zbyt wysokie z wartością 0.1406. Wartości dla poszczególnych kategorii również nie zachwycają.

In [320]:
calculate_metrics(passages, find_named_entities_spacy, perform_few_shot_prompt, True)

{'persName': {'precision': 0.21739130434782608,
  'recall': 0.09715025906735751,
  'f1_score': 0.1342882721575649},
 'orgName': {'precision': 0.22304832713754646,
  'recall': 0.13043478260869565,
  'f1_score': 0.1646090534979424},
 'placeName': {'precision': 0.2485207100591716,
  'recall': 0.0594059405940594,
  'f1_score': 0.0958904109589041},
 'geogName': {'precision': 0.08333333333333333,
  'recall': 0.00819672131147541,
  'f1_score': 0.01492537313432836},
 'date': {'precision': 0.32786885245901637,
  'recall': 0.08695652173913043,
  'f1_score': 0.13745704467353953},
 'time': {'precision': 0.0, 'recall': 0.0, 'f1_score': 0},
 'total': {'precision': 0.22811671087533156,
  'recall': 0.09368191721132897,
  'f1_score': 0.1328185328185328}}

|           | persName | orgName | placeName | geogName | date   | time | **total**  |
|-----------|----------|---------|-----------|----------|--------|------|------------|
| precision | 0.2173   | 0.2230  | 0.2485    | 0.0833   | 0.3278 | 0.0  | **0.2281** |
| recall    | 0.0971   | 0.1304  | 0.0594    | 0.0081   | 0.0869 | 0.0  | **0.0936** |
| f1_score  | 0.1342   | 0.1646  | 0.0958    | 0.0149   | 0.1374 | 0.0  | **0.1328** |

Użycie zapytania `few-shot` poprawia ogólne `precision` o kilka punktów procentowych, ale osłabia o podobną wartość `recall`, przez co `f1-score` wychodzi odrobinę mniejsze.

Generalnie mniejszy `f1-score` dla metody `few-shot` może zaskakiwać, lecz przytoczone poniżej przykłady pokazują potencjalne źródło problemów.

Sprawdzę poniżej, czy poluzowanie kryteriów dopasowania przyniesie poprawę.

In [321]:
calculate_metrics(passages, find_named_entities_spacy, perform_zero_shot_prompt, False)

{'persName': {'precision': 0.2063157894736842,
  'recall': 0.12694300518134716,
  'f1_score': 0.15717722534081793},
 'orgName': {'precision': 0.24726775956284153,
  'recall': 0.19546436285097193,
  'f1_score': 0.21833534378769603},
 'placeName': {'precision': 0.28865979381443296,
  'recall': 0.15841584158415842,
  'f1_score': 0.20456621004566208},
 'geogName': {'precision': 0.03731343283582089,
  'recall': 0.04,
  'f1_score': 0.0386100386100386},
 'date': {'precision': 0.4339622641509434,
  'recall': 0.39655172413793105,
  'f1_score': 0.41441441441441446},
 'time': {'precision': 0.013157894736842105,
  'recall': 1.0,
  'f1_score': 0.025974025974025976},
 'total': {'precision': 0.22637159981558322,
  'recall': 0.1775768535262206,
  'f1_score': 0.19902715849209565}}

In [322]:
calculate_metrics(passages, find_named_entities_spacy, perform_few_shot_prompt, False)

{'persName': {'precision': 0.2753623188405797,
  'recall': 0.12305699481865284,
  'f1_score': 0.17009847806624886},
 'orgName': {'precision': 0.29182156133828996,
  'recall': 0.17065217391304346,
  'f1_score': 0.21536351165980797},
 'placeName': {'precision': 0.3254437869822485,
  'recall': 0.07779349363507779,
  'f1_score': 0.12557077625570773},
 'geogName': {'precision': 0.08333333333333333,
  'recall': 0.00819672131147541,
  'f1_score': 0.01492537313432836},
 'date': {'precision': 0.7213114754098361,
  'recall': 0.19130434782608696,
  'f1_score': 0.302405498281787},
 'time': {'precision': 0.0, 'recall': 0.0, 'f1_score': 0},
 'total': {'precision': 0.3112290008841733,
  'recall': 0.12781408859840232,
  'f1_score': 0.1812097812097812}}

Już pobieżnie rzucając okiem na wyniki widać, że uległy one poprawie, zarówno generalnie, jak i dla poszczególnych kategorii. Wynika z tego, że mogą istnieć drobne różnice w odnalezionych nazwach własnych, które uniemożliwiają bezpośrednie dopasowanie.

Poza rozłącznymi listami odnalezionych nazw własnych dla poszczególnych klas, istnieje wiele innych przykładów, które pokazują przyczynę problemu niskich metryk oceniających jakość działania:

`zero-show`:
1. persName:
- baseline: ['G.', 'Edward Griffin'], LLM: 'G. Edward Griffin' - SpaCy rozdziela skrót imienia od reszty

2. orgName:
- baseline: 'Narodowego Instytutu Zdrowia](https://www.ncbi.nlm.nih.gov', LLM: 'Narodowy Instytut Zdrowia' - SpaCy dokleja kawałek URL do nazwy
- baseline: 'Discover Bank CD', LLM: 'Discover Bank` - LLM obcina część nazwy
- baseline: 'EBSA ds.', LLM: 'EBSA' - SpaCy dokleja kolejne słowa do nazwy

3. placeName:
- baseline: 'Japonii', LLM: 'Japonia' - LLM zmienia na mianownik pomimo wyraźnych instrukcji, żeby tego nie robić

4. date:
- baseline: 'sierpniu', LLM: 'sierpniowe' - różne tokeny rozpoznane, a istnieje tylko ten z baseline, LLM dziwnie zmienia odmianę
- baseline: [], LLM: 'dowolny dzień' - LLM rozpoznaje określenie daty, ale to nie jest data

5. Różne klasy
- baseline['orgName'] = 'AHS', LLM['geogName'] = 'AHS' - dla SpaCy AHS jest organizacją, a dla LLM nazwą geograficzną. W tym wypadku SpaCy ma rację.

6. Uznanie słów nie będących nazwą własną za takową:
LLM: 'twoja firma'

`few-show`:
1. persName:
- baseline: 'Obamy', LLM: 'Obama' - LLM zmienia na mianownik pomimo wyraźnych instrukcji, żeby tego nie robić

2. orgName:
- baseline: 'Citi Bankiem', LLM: 'Citi Bank' - LLM zmienia na mianownik pomimo wyraźnych instrukcji, żeby tego nie robić
- baseline: 'The National Counseling Society Adres', LLM: 'The National Counseling Society' - SpaCy uwzglęnia więcej słów jako nazwę własną niż LLM

3. placeName:
- baseline: 'Kanadzie', LLM: 'Kanada' - LLM zmienia na mianownik pomimo wyraźnych instrukcji, żeby tego nie robić
- baseline: 'Frankfurcie', LLM: 'Frankfurt' - podobnie, jak wyżej

4. date:
- baseline: '2006r.', LLM: '2006' - uznanie 'r.' za część daty lub nie

Przypadki były zbierane na różnych fragmentach danych, stąd są one inne dla `zero-shot` i `few-shot`, ale widać powtarzalność zachowań.

Uderza dość spore grono przypadków dla `few-shot`, gdzie LLM zmienił odmianę słowa, a robiąc to zignorował zalecenie podane w prompcie. Używany model ograniczony jest do wyłącznie siedmiu miliardów parametrów, a prompt oscyluje przy maksymalnej granicy długości, więc szansa, że model nie uwzględni wszystkich instrukcji zawartych w prompcie rośnie.

Generalnie nie można powiedzieć, żeby wyniki od SpaCy były dużo lepsze od tych generowanych przez LLM, co podważa zasadność używania SpaCy jako baseline.

Czasy na przejście po wszystkich pasażach:

Zero-Shot: 3200.7369158267975s ~53:20
Few-Shot: 3537.733955144882s ~58:57

### NER na danych ground truth

W momencie opracowywania wyników we wspólnym arkuszu udało się zebrać 97 fragmentów tekstu z wyodrębnionymi nazwami własnymi. Arkusz ten wymagał dalszego przygotowania, postaci:
- ujednolicenie zapisu odnalezionych nazw własnych i ich klas
- sprowadzenie wszystkich słówek do odmiany występującej w tekście (lematyzację uznaję za dodatkowy krok, który nie jest wymagany w tym ćwiczeniu)
- drobne poprawki przypisanych klas

Nie przeprowadzałem gruntownej rewizji i ujednolicenia zebranych wyników. Ważnym była dla mnie spójność formatu oraz odmiany odnalezionych nazw własnych. W związku z tym należy się liczyć z kilkoma niespójnościami pomiędzy poszczególnymi elementami ze zbioru, które ważą na jakości dokonanej operacji NER.

Poniżej funkcja do wczytania danych z pliku i zmapowania uniwersalnych znaczników klas nazw własnych, do tych ze SpaCy, których używam w ćwiczeniu:

In [301]:
import ast
import csv

cat_map = {   
    'per': 'persName',
    'org': 'orgName',
    'gpe': 'placeName',
    'loc': 'geogName',
    'date': 'date',
    'time': 'time'
 }

def readInGroundTruthSents():
    with open('LLM_NER_anotacje-mod-2.csv', mode='r', newline='') as file:
        results = {}
        sents = []
        reader = csv.DictReader(file)
        for row in reader:
            sentence = row['Tekst']
            sents.append(sentence)
            named_entities = {}
            entities = ast.literal_eval(row['Encje'])
            for entity in entities:
                entity_name = entity[0]
                entity_cat = entity[1]
                entity_cat_mapped = cat_map[entity_cat.lower()]
                if entity_cat_mapped not in named_entities.keys():
                    named_entities[entity_cat_mapped] = []
                named_entities[entity_cat_mapped].append(entity_name)
            results[sentence] = named_entities
        return sents, results            

In [302]:
groundTruthSents, groundTruthNer  = readInGroundTruthSents()

def getGroundTruthNer(passage):
    return groundTruthNer[passage]

In [323]:
calculate_metrics(groundTruthSents, getGroundTruthNer, perform_zero_shot_prompt, True)

{'persName': {'precision': 0.543859649122807,
  'recall': 0.4492753623188406,
  'f1_score': 0.49206349206349204},
 'orgName': {'precision': 0.6619718309859155,
  'recall': 0.34814814814814815,
  'f1_score': 0.4563106796116505},
 'placeName': {'precision': 0.3333333333333333,
  'recall': 0.21348314606741572,
  'f1_score': 0.2602739726027397},
 'geogName': {'precision': 0.0, 'recall': 0.0, 'f1_score': 0},
 'date': {'precision': 0.6086956521739131,
  'recall': 0.56,
  'f1_score': 0.5833333333333334},
 'time': {'precision': 0.0, 'recall': 0.0, 'f1_score': 0},
 'total': {'precision': 0.5235849056603774,
  'recall': 0.3373860182370821,
  'f1_score': 0.41035120147874315}}

|           | persName | orgName | placeName | geogName | date   | time | **total**  |
|-----------|----------|---------|-----------|----------|--------|------|------------|
| precision | 0.5438   | 0.6619  | 0.3333    | 0.0      | 0.6086 | 0.0  | **0.5235** |
| recall    | 0.4492   | 0.3481  | 0.2134    | 0.0      | 0.5600 | 0.0  | **0.3373** |
| f1_score  | 0.4920   | 0.4563  | 0.2602    | 0.0      | 0.5833 | 0.0  | **0.4103** |

Już widać, że jakość wyników dostarczonych przez LLM dla dobrze oznaczonych nazw własnych jest dużo lepsza, niż dla tych porównywanych ze SpaCy. Zarówno wartości ogólne, jak i dla poszczególnych kategorii są 2-3x lepsze od tych z poprzedniej sekcji laboratorium.

Nie chcąc produkować zbyt dużo treści w tym sprawozdaniu pomijam wartości metryk dla poszczególnych klas i przy porównaniu poszczególnych podejść skupię się na wartościach generalnych, które dla `zero-shot` wynoszą odpowiednio: `precision = 0.5235`, `recall = 0.3373` i `f1-score = 0.4103`.

In [324]:
calculate_metrics(groundTruthSents, getGroundTruthNer, perform_few_shot_prompt, True)

{'persName': {'precision': 0.56,
  'recall': 0.4057971014492754,
  'f1_score': 0.4705882352941177},
 'orgName': {'precision': 0.6176470588235294,
  'recall': 0.3111111111111111,
  'f1_score': 0.41379310344827586},
 'placeName': {'precision': 0.6111111111111112,
  'recall': 0.3626373626373626,
  'f1_score': 0.4551724137931034},
 'geogName': {'precision': 0, 'recall': 0.0, 'f1_score': 0},
 'date': {'precision': 0.5714285714285714,
  'recall': 0.3333333333333333,
  'f1_score': 0.4210526315789474},
 'time': {'precision': 0.0, 'recall': 0.0, 'f1_score': 0},
 'total': {'precision': 0.5873015873015873,
  'recall': 0.33636363636363636,
  'f1_score': 0.4277456647398844}}

|           | persName | orgName | placeName | geogName | date   | time | **total**  |
|-----------|----------|---------|-----------|----------|--------|------|------------|
| precision | 0.56     | 0.6176  | 0.6111    | 0.0      | 0.5714 | 0.0  | **0.5873** |
| recall    | 0.4057   | 0.3111  | 0.3626    | 0.0      | 0.3333 | 0.0  | **0.3363** |
| f1_score  | 0.4705   | 0.4137  | 0.4551    | 0.0      | 0.4210 | 0.0  | **0.4277** |

Tym razem zapytanie `few-shot` dostarcza lepszych wyników, które przewyżsa `zero-shot` z wartością 0.5863 dla `precision` o 6 punktów procentowych, ma bardzo zbliżony `recall = 0.3363`, a więc i `f1-score` jest większy i wynosi 0.4277, czyli 1.5pp więcej.

`few-shot` będący bardziej dokładną formą prompta z reguły powinien dostarczać lepszych wyników, niż `zero-shot`, i tak też się dzieje w tym przypadku. Ewidentnie mniejsza jakość `few-shot` dla korpusu FIQA wynika z problemów samego baseline-u.

In [325]:
calculate_metrics(groundTruthSents, getGroundTruthNer, find_named_entities_spacy, True)

{'persName': {'precision': 0.6363636363636364,
  'recall': 0.6086956521739131,
  'f1_score': 0.6222222222222223},
 'orgName': {'precision': 0.6704545454545454,
  'recall': 0.4338235294117647,
  'f1_score': 0.5267857142857143},
 'placeName': {'precision': 0.6736842105263158,
  'recall': 0.6956521739130435,
  'f1_score': 0.6844919786096257},
 'geogName': {'precision': 0.75, 'recall': 0.5, 'f1_score': 0.6},
 'date': {'precision': 0.41379310344827586,
  'recall': 0.48,
  'f1_score': 0.4444444444444445},
 'time': {'precision': 0.0, 'recall': 0.0, 'f1_score': 0},
 'total': {'precision': 0.6293706293706294,
  'recall': 0.5405405405405406,
  'f1_score': 0.5815831987075929}}

|           | persName | orgName | placeName | geogName | date   | time | **total**  |
|-----------|----------|---------|-----------|----------|--------|------|------------|
| precision | 0.6363   | 0.6704  | 0.6736    | 0.75     | 0.4137 | 0.0  | **0.6293** |
| recall    | 0.6086   | 0.4338  | 0.6956    | 0.5      | 0.4800 | 0.0  | **0.5405** |
| f1_score  | 0.6222   | 0.5267  | 0.6844    | 0.6      | 0.4444 | 0.0  | **0.5815** |

Okazuje się, że pomimo niewystarczającej jakości SpaCy do zastosowania jako baseline i kilku gorzkich słów pod jego adresem, wyniki uzyskane dla ręcznie otagowanych danych tą metodą przewyższają obydwa podejścia LLM we wszystkich metrykach: `precision = 0.6293`, `recall = 0.5405` i `f1-score = 0.5815`.

Różnica jest znacząca i wyraźnie pokazuje, że naiwne modele wciąż mogą okazać się lepszym rozwiązaniem, niż modele LLM o niewystarczającej pojemności, lub zbyt generalnym przeszkoleniu.

Jak i poprzednio uruchamiam odpowiedniki powyższych funkcji z luźniejszą metodą dopasowywania nazw własnych i znów oczekuję poprawy metryk.

In [305]:
calculate_metrics(groundTruthSents, getGroundTruthNer, perform_zero_shot_prompt, False)

{'persName': {'precision': 0.6666666666666666,
  'recall': 0.5507246376811594,
  'f1_score': 0.603174603174603},
 'orgName': {'precision': 0.8028169014084507,
  'recall': 0.4222222222222222,
  'f1_score': 0.5533980582524272},
 'placeName': {'precision': 0.49122807017543857,
  'recall': 0.3146067415730337,
  'f1_score': 0.3835616438356165},
 'geogName': {'precision': 0.0, 'recall': 0.0, 'f1_score': 0},
 'date': {'precision': 0.9130434782608695,
  'recall': 0.84,
  'f1_score': 0.8749999999999999},
 'time': {'precision': 0.3333333333333333, 'recall': 0.2, 'f1_score': 0.25},
 'total': {'precision': 0.6839622641509434,
  'recall': 0.44072948328267475,
  'f1_score': 0.5360443622920518}}

In [306]:
calculate_metrics(groundTruthSents, getGroundTruthNer, perform_few_shot_prompt, False)

{'persName': {'precision': 0.68,
  'recall': 0.4927536231884058,
  'f1_score': 0.5714285714285715},
 'orgName': {'precision': 0.75,
  'recall': 0.37777777777777777,
  'f1_score': 0.5024630541871921},
 'placeName': {'precision': 0.7037037037037037,
  'recall': 0.4175824175824176,
  'f1_score': 0.5241379310344827},
 'geogName': {'precision': 0, 'recall': 0.0, 'f1_score': 0},
 'date': {'precision': 0.7857142857142857,
  'recall': 0.4583333333333333,
  'f1_score': 0.5789473684210527},
 'time': {'precision': 1.0, 'recall': 0.6, 'f1_score': 0.7499999999999999},
 'total': {'precision': 0.7248677248677249,
  'recall': 0.41515151515151516,
  'f1_score': 0.5279383429672446}}

In [317]:
calculate_metrics(groundTruthSents, getGroundTruthNer, find_named_entities_spacy, False)

{'persName': {'precision': 0.803030303030303,
  'recall': 0.7681159420289855,
  'f1_score': 0.785185185185185},
 'orgName': {'precision': 0.8181818181818182,
  'recall': 0.5294117647058824,
  'f1_score': 0.6428571428571428},
 'placeName': {'precision': 0.6947368421052632,
  'recall': 0.717391304347826,
  'f1_score': 0.7058823529411765},
 'geogName': {'precision': 0.75, 'recall': 0.5, 'f1_score': 0.6},
 'date': {'precision': 0.7586206896551724,
  'recall': 0.88,
  'f1_score': 0.8148148148148148},
 'time': {'precision': 0.5, 'recall': 0.4, 'f1_score': 0.4444444444444445},
 'total': {'precision': 0.7622377622377622,
  'recall': 0.6546546546546547,
  'f1_score': 0.7043618739903069}}

Faktycznie, wartości każej metryki są dużo lepsze. Znajdźmy kilka przykładów dla każdej z metody, które obrazowałyby przyczynę takiego stanu rzeczy.

`zero-shot`:
1. persName:
- baseline: "Mika'e Tysona" LLM: "Mike Tyson" - sprowadzenie nazwy przez LLM do formy podstawowej
- baseline" "Bosak" LLM: brak rozpoznania nazwiska

2. orgName:
- baseline: "Reuters", LLM: "Agencja Reuters" - tu chyba LLM miał rację i to baseline jest niedokładnie wyspecyfikowany


3. placeName:
- baseline: "Teksasie" LLM: "Teksas" - odmiana przez LLM
- baseline: "Syrii", LLM: "środkowa część Syrii" - zbyt duży fragment tekstu uznany za nazwę własną przez LLM

4. dateName:
- baseline: 'poniedziałek (9 grudnia)', LLM: '9 grudnia' - pominięcie dnia tygodnia przez LLM

`few-shot`:

Generalnie to przykłady dla `few-shot` są bardzo zbliżone do tych dla `zero-shot`, więc podam tu tylko jeden odstający przykład:

1. persName:
- baseline: "Mika'e Tysona" LLM: "Mike'e Tyson" - taka dziwna odmiana zaproponowana przez LLM 

Nie widać tego w podanych przykładach, ale dominującą formą rozbieżności pomiędzy wynikiem LLM (przy obydwu rodzajach prompta), a "ground truth" jest sprowadzanie nazw do mianownika przez LLM. Usunięcie tego mankamentu LLM dałoby dużo lepsze wyniki. 

`SpaCy`

1. persName:
- baseline: "Steve'a Jobs'a", SpaCy: ["Steve'a", "Jobs'a"] - oddzielenie imienia od nazwiska

2. orgName:
- Spacy: "Inaki Pena" - uznany za organizację
- baseline: "America PAC", SpaCy: "Muska komitetu America PAC" - zbyt dużo uznane za nazwę własną

3. placeName:
- baseline: "Teksasie", SpaCy: "Teksasie Jake Paul" - dołączenie imienia i nazwiska do nazwy stanu

4. date:
- baseline: "2023", Spacy: "2023 roku" - tu SpaCy okazało się lepsze od baseline-u

SpaCy robi róznego rodzaju błędy, czasem źle rozdzieli nazwę własną, a czasem połączy zbyt wiele słów jako pojedyńczą nazwę własną, bądź też pomyli kategorię rozważanego słowa. Błędy te są dość rozbieżne i niszowe, przez co dla badanych przykładów SpaCy pokonuje LLM, który bardzo często niepotrzebnie sprowadza dobrze odnalezioną nazwę własną do mianownika - wystarczyłoby przekonać LLM, może lepszym promptem lub pojemniejszym modelem, do pozostawiania słów takimi, jakimi są, żeby to LLM wysforował się na pierwsze miejsce.



#### Porównanie NER do LLM NER

Czas, pamięć, miejsce na HDD, miejsce w pamięci i obciążenie CPU

Zachowanie odmiany, bo lematyzacja jest osobnym krokiem - zwłaszcza możliwy problem z nazwami własnymi:
- Wisła - Wiśle
- Polska - Polski

Co z zapisem lat: 2017 vs 2017 r. a może 2025 roku
Skomplikowany zapis: poniedziałek (9 grudnia)
również dla nazw własnych: Rady Biznesu USA-Chiny (USCBC)
rzecznik Prokuratury Okręgowej w Warszawie, prok. Piotr Skiba
telewizja TVN czy samo TVN, czy też linii Wizz Air czy samo Wizz Air